In [215]:
import pandas as pd
import re
import nltk
import numpy as np
import pandas as pd
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from termcolor import colored
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss, confusion_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc , precision_score , classification_report
from sklearn.linear_model import LogisticRegression
#from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.linear_model import SGDClassifier
#from imblearn.over_sampling import SMOTE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")
#from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from scipy.stats import stats

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\snake\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\snake\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [216]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
#from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

#from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [271]:
data = pd.read_csv('train.csv')

In [272]:
data['tweet'] = data['tweet'].apply(lambda row: str(row))
data

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1
...,...,...,...
7269,3343,@mention Google plze Tammi. I'm in middle of ...,1
7270,5334,RT @mention ÷¼ Are you all set? ÷_ {link} ÷...,1
7271,5378,RT @mention Aha! Found proof of lactation room...,1
7272,2173,We just launched our iPad app at #SXSW! Get al...,1


In [273]:
def preprocess(x):
    x = x.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("don't", "do not")\
                           .replace("%", " percent ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace(";", " ").replace("&", " ")
    return x

In [274]:
data['tweet'] = data['tweet'].fillna("").apply(preprocess)

In [275]:
# Setting stopwords
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.remove("not")
STOPWORDS.remove('no')
STOPWORDS.remove('against')
STOPWORDS.remove('because')
STOPWORDS.remove('while')
STOPWORDS.remove('why')
STOPWORDS.remove('how')
STOPWORDS.remove('can')
STOPWORDS.add('quot')
STOPWORDS.add('RT')
STOPWORDS.add('sxsw')
STOPWORDS.add('SXSW')
STOPWORDS.add('link')
STOPWORDS.add('The')
STOPWORDS.add('Apple')
STOPWORDS.add('Google')
STOPWORDS.add('iPad')
# Function to expand tweet
def expand_tweet(tweet):
    expanded_tweet = []
    for word in tweet:
        if re.search("n't", word):
            expanded_tweet.append(word.split("n't")[0])
            expanded_tweet.append("not")
        else:
            expanded_tweet.append(word)
    return expanded_tweet

# Function to process tweets
def clean_tweet(data, wordNetLemmatizer, porterStemmer):
    data['Clean_tweet'] = data['tweet']
    print(colored("Removing user handles starting with @", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].str.replace("@[\w]*","")
    print(colored("Removing numbers and special characters", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].str.replace("[^a-zA-Z' ]","")
    print(colored("Removing urls", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].replace(re.compile(r"((www\.[^\s]+)|(https?://[^\s]+))"), "")
    print(colored("Removing single characters", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].replace(re.compile(r"(^| ).( |$)"), " ")
    print(colored("Tokenizing", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].str.split()
    print(colored("Removing stopwords", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: [word for word in tweet if word not in STOPWORDS])
    print(colored("Expanding not words", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: expand_tweet(tweet))
    print(colored("Lemmatizing the words", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: [wordNetLemmatizer.lemmatize(word) for word in tweet])
    print(colored("Stemming the words", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: [porterStemmer.stem(word) for word in tweet])
    print(colored("Combining words back to tweets", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: ' '.join(tweet))
    return data

# Define processing methods
wordNetLemmatizer = WordNetLemmatizer()
porterStemmer = PorterStemmer()

# Pre-processing the tweets
print(colored("Processing train data", "green"))
train_data = clean_tweet(data, wordNetLemmatizer, porterStemmer)


Processing train data
Removing user handles starting with @
Removing numbers and special characters
Removing urls
Removing single characters
Tokenizing
Removing stopwords
Expanding not words
Lemmatizing the words
Stemming the words
Combining words back to tweets


In [276]:
train_data

,tweet_id,tweet,sentiment,Clean_tweet
0,1701,#sxswnui #sxsw #apple defining language of tou...,1,sxswnui appl defin languag touch differ dialec...
1,1851,Learning ab Google doodles! All doodles should...,1,learn ab doodl all doodl light funni amp innov...
2,2689,one of the most in-your-face ex. of stealing t...,2,one inyourfac ex steal show yr At school mkt e...
3,4525,This iPhone #SXSW app would b pretty awesome i...,0,thi iphon app would pretti awesom not crash ev...
4,3604,Line outside the Apple store in Austin waiting...,1,line outsid store austin wait new
...,...,...,...,...
7269,3343,@mention Google plze Tammi. I'm in middle of ...,1,plze tammi i'm middl crazi everyth soooooo busi
7270,5334,RT @mention ÷¼ Are you all set? ÷_ {link} ÷...,1,are set edchat musedchat sxswi newtwitt
7271,5378,RT @mention Aha! Found proof of lactation room...,1,aha found proof lactat room excus mother' room...
7272,2173,We just launched our iPad app at #SXSW! Get al...,1,We launch app get detail first edit free


In [277]:
tfidf = TfidfVectorizer( ngram_range=(1,2))
kk = tfidf.fit_transform(train_data.Clean_tweet)

In [278]:
kk

<7274x32695 sparse matrix of type '<class 'numpy.float64'>'
	with 59596 stored elements in Compressed Sparse Row format>

In [279]:
# clf = SGDClassifier(alpha=0.0001, penalty='l2', loss='log',)
# clf.fit(kk,train_data.sentiment)
# sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
# sig_clf.fit(kk,train_data.sentiment)

In [280]:
from sklearn.linear_model import LogisticRegression
sig_clf = LogisticRegression(C=1000)
sig_clf.fit(kk,train_data.sentiment)

LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [281]:
testdata = pd.read_csv('test.csv')
testdata

,tweet_id,tweet
0,7506,Audience Q: What prototyping tools do you use?...
1,7992,At SXSW? Send Your Best Photos &amp; Videos to...
2,247,@mention and here's a pic of you winning your...
3,7688,Google Marissa Mayer: mobile phone as a cursor...
4,3294,#SXSW Google maps is even cooler than I thought
...,...,...
1814,1550,@mention @mention @mention Hmmm....how fast ca...
1815,1933,Samsung Galaxy S II Appears At FCC And Team An...
1816,9052,@mention You could buy a new iPad 2 tmrw at th...
1817,4219,Wow very long queue of people at apple pop up ...


# Preprocessing test data

In [282]:
def preprocess(x):
    x = x.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("don't", "do not")\
                           .replace("%", " percent ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace(";", " ").replace("&", " ")
    return x

In [283]:
test_data['tweet'] = test_data['tweet'].fillna("").apply(preprocess)

In [284]:
# Setting stopwords
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.remove("not")
STOPWORDS.remove('no')
STOPWORDS.remove('against')
STOPWORDS.remove('because')
STOPWORDS.remove('while')
STOPWORDS.remove('why')
STOPWORDS.remove('how')
STOPWORDS.remove('can')
STOPWORDS.add('quot')
STOPWORDS.add('RT')
STOPWORDS.add('sxsw')
STOPWORDS.add('SXSW')
STOPWORDS.add('link')
STOPWORDS.add('The')
STOPWORDS.add('Apple')
STOPWORDS.add('Google')
STOPWORDS.add('iPad')

# Function to expand tweet
def expand_tweet(tweet):
    expanded_tweet = []
    for word in tweet:
        if re.search("n't", word):
            expanded_tweet.append(word.split("n't")[0])
            expanded_tweet.append("not")
        else:
            expanded_tweet.append(word)
    return expanded_tweet

# Function to process tweets
def clean_tweet(data, wordNetLemmatizer, porterStemmer):
    data['Clean_tweet'] = testdata['tweet']
    print(colored("Removing user handles starting with @", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].str.replace("@[\w]*","")
    print(colored("Removing numbers and special characters", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].str.replace("[^a-zA-Z' ]","")
    print(colored("Removing urls", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].replace(re.compile(r"((www\.[^\s]+)|(https?://[^\s]+))"), "")
    print(colored("Removing single characters", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].replace(re.compile(r"(^| ).( |$)"), " ")
    print(colored("Tokenizing", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].str.split()
    print(colored("Removing stopwords", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: [word for word in tweet if word not in STOPWORDS])
    print(colored("Expanding not words", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: expand_tweet(tweet))
    print(colored("Lemmatizing the words", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: [wordNetLemmatizer.lemmatize(word) for word in tweet])
    print(colored("Stemming the words", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: [porterStemmer.stem(word) for word in tweet])
    print(colored("Combining words back to tweets", "yellow"))
    data['Clean_tweet'] = data['Clean_tweet'].apply(lambda tweet: ' '.join(tweet))
    return data

# Define processing methods
wordNetLemmatizer = WordNetLemmatizer()
porterStemmer = PorterStemmer()

# Pre-processing the tweets
print(colored("Processing train data", "green"))
test_data = clean_tweet(testdata, wordNetLemmatizer, porterStemmer)

Processing train data
Removing user handles starting with @
Removing numbers and special characters
Removing urls
Removing single characters
Tokenizing
Removing stopwords
Expanding not words
Lemmatizing the words
Stemming the words
Combining words back to tweets


In [285]:
test_data

,tweet_id,tweet,Clean_tweet
0,7506,Audience Q: What prototyping tools do you use?...,audienc what prototyp tool use sketchbooksshar...
1,7992,At SXSW? Send Your Best Photos &amp; Videos to...,At send your best photo amp video citizenjourn...
2,247,@mention and here's a pic of you winning your...,here' pic win ipad unsix cc cont
3,7688,Google Marissa Mayer: mobile phone as a cursor...,marissa mayer mobil phone cursor physic locat ...
4,3294,#SXSW Google maps is even cooler than I thought,map even cooler thought
...,...,...,...
1814,1550,@mention @mention @mention Hmmm....how fast ca...,hmmmhow fast can appl build new store time
1815,1933,Samsung Galaxy S II Appears At FCC And Team An...,samsung galaxi II appear At fcc and team andro...
1816,9052,@mention You could buy a new iPad 2 tmrw at th...,you could buy new tmrw popup store
1817,4219,Wow very long queue of people at apple pop up ...,wow long queue peopl appl pop store bought ipa...


In [286]:
test_tfidf = test_data.Clean_tweet

In [287]:
test_tfidf = tfidf.transform(test_tfidf)
test_tfidf

<1819x32695 sparse matrix of type '<class 'numpy.float64'>'
	with 8744 stored elements in Compressed Sparse Row format>

In [288]:
ytest= sig_clf.predict(test_tfidf)

In [289]:
sent = pd.DataFrame(ytest,columns=['sentiment'])

In [290]:
result = pd.concat([test_data['tweet_id'],pd.Series(ytest)],axis=1)
result.columns = ['tweet_id','sentiment']
result

,tweet_id,sentiment
0,7506,1
1,7992,1
2,247,1
3,7688,2
4,3294,1
...,...,...
1814,1550,1
1815,1933,1
1816,9052,1
1817,4219,1


In [291]:
result.to_csv('final1.csv',index=False)